In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, f1_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

def load_models():
    """Load trained models."""
    models = {}
    model_paths = {
        'preprocessor': '../models/preproc.joblib',
        'temp_model': '../models/xgb_temp.joblib',
        'time_model': '../models/xgb_time.joblib',
        'method_model': '../models/xgb_method.joblib'
    }
    
    for name, path in model_paths.items():
        try:
            models[name] = joblib.load(path)
            print(f"✅ Loaded {name}")
        except Exception as e:
            print(f"❌ Failed to load {name}: {e}")
    
    return models

def evaluate_models(models, df):
    """Evaluate all models on the dataset."""
    print("Model Evaluation Results")
    print("=" * 50)
    
    # Prepare features
    feature_cols = [
        'Metal source (mmol)', 'Linker(s) (mmol)', 'Solvent(s) (mL)', 
        'Modulator / Additive', 'Pressure'
    ]
    X = df[feature_cols]
    
    # Extract numerical targets
    temp_values = []
    for temp_str in df['Temp (°C)']:
        if pd.isna(temp_str) or temp_str == '':
            temp_values.append(np.nan)
        else:
            numbers = re.findall(r'\d+\.?\d*', str(temp_str))
            temp_values.append(float(numbers[0]) if numbers else np.nan)
    
    time_values = []
    for time_str in df['Time (h)']:
        if pd.isna(time_str) or time_str == '':
            time_values.append(np.nan)
        else:
            numbers = re.findall(r'\d+\.?\d*', str(time_str))
            time_values.append(float(numbers[0]) if numbers else np.nan)
    
    y_temp = pd.Series(temp_values)
    y_time = pd.Series(time_values)
    y_method = df['Method']
    
    # Evaluate temperature model
    if 'temp_model' in models:
        temp_pred = models['temp_model'].predict(X)
        temp_mae = mean_absolute_error(y_temp.dropna(), temp_pred[~y_temp.isna()])
        temp_rmse = np.sqrt(mean_squared_error(y_temp.dropna(), temp_pred[~y_temp.isna()]))
        print(f"🌡️ Temperature Model:")
        print(f"   MAE: {temp_mae:.2f}°C")
        print(f"   RMSE: {temp_rmse:.2f}°C")
        print()
    
    # Evaluate time model
    if 'time_model' in models:
        time_pred = models['time_model'].predict(X)
        time_mae = mean_absolute_error(y_time.dropna(), time_pred[~y_time.isna()])
        time_rmse = np.sqrt(mean_squared_error(y_time.dropna(), time_pred[~y_time.isna()]))
        print(f"⏰ Time Model:")
        print(f"   MAE: {time_mae:.2f} hours")
        print(f"   RMSE: {time_rmse:.2f} hours")
        print()
    
    # Evaluate method model
    if 'method_model' in models:
        method_pred = models['method_model'].predict(X)
        method_acc = accuracy_score(y_method, method_pred)
        method_f1 = f1_score(y_method, method_pred, average='weighted')
        print(f"🔬 Method Model:")
        print(f"   Accuracy: {method_acc:.3f}")
        print(f"   F1 Score: {method_f1:.3f}")
        print()
        
        # Classification report
        print("Classification Report:")
        print(classification_report(y_method, method_pred))

# Load data and models
df = pd.read_csv('../data/processed/mof_runs.csv')
models = load_models()

print(f"Dataset loaded with {len(df)} rows")
print(f"Models loaded: {list(models.keys())}")
print()

# Evaluate models
evaluate_models(models, df)

# Dataset statistics
print("\nDataset Statistics")
print("=" * 50)
print(f"Total samples: {len(df)}")
print(f"Unique MOFs: {df['Target'].nunique()}")
print(f"Unique methods: {df['Method'].nunique()}")
print(f"Unique pressures: {df['Pressure'].nunique()}")
print()

print("Method distribution:")
print(df['Method'].value_counts())
print()

print("Pressure distribution:")
print(df['Pressure'].value_counts())
